In [158]:
# Using feast to create feature store for the repo
#install feat

!pip install feast
!pip freeze | echo feast

feast


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\asokk\AppData\Roaming\Python\Python38\site-packages\pip\_internal\cli\base_command.py", line 180, in _main
    status = self.run(options, args)
  File "C:\Users\asokk\AppData\Roaming\Python\Python38\site-packages\pip\_internal\commands\freeze.py", line 103, in run
    sys.stdout.write(line + '\n')
OSError: [Errno 22] Invalid argument


In [159]:
#%%bash
#cd ../
#feast init --minimal --template local features
#cd features
#touch features.py

In [160]:
import pandas as pd
from pathlib import Path
import config

In [181]:
# Loading features to data frame
feature_path = Path(config.DATA_DIR, "boston.csv")

features = pd.read_csv(feature_path, parse_dates=['CREATED_TIME'])

#features.CREATED_TIME = features.CREATED_TIME.apply(lambda x: str(x))

#features['CREATED_TIME'] = features['CREATED_TIME'].astype(datetime)

print(features.head())

features.info()

   ID     CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0   0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1   1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2   2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3   3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4   4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   

   PTRATIO       B  LSTAT  MEDV        CREATED_TIME  
0     15.3  396.90   4.98  24.0 2020-09-07 19:23:11  
1     17.8  396.90   9.14  21.6 2020-11-22 20:11:49  
2     17.8  392.83   4.03  34.7 2020-10-02 11:06:31  
3     18.7  394.63   2.94  33.4 2020-09-06 00:16:44  
4     18.7  396.90   5.33  36.2 2020-07-12 14:38:45  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            5

In [182]:
# Convert to parquet format as Feast only support it thorugh offline

features.to_parquet(
    Path(config.DATA_DIR, "boston.parquet"),
    compression=None,
    allow_truncated_timestamps=True
)

Feature Store


In [183]:
from feast import Entity, Feature, FeatureView, ValueType
from feast.data_source import FileSource
from google.protobuf.duration_pb2 import Duration
from datetime import datetime

define the location of the data source to feast

In [185]:
START_TIME = "2021-07-23"
feature_source = FileSource(
    path=str(Path(config.DATA_DIR, "boston.parquet")),
    event_timestamp_column="CREATED_TIME"
)

Nextly, define the main entity of feature data points. In our case, each data point has a unique ID.

In [186]:
feature_entity = Entity(
    name="ID",
    value_type=ValueType.INT64,
    description="house id"
)

Finally, create the Feature View that loads specific features of valrious value types from the feature source for a specific period of time

In [187]:
feature_details_view = FeatureView(
    name="house_price_details",
    entities=["ID"],
     ttl=Duration(
        seconds=(datetime.today() - datetime.strptime(START_TIME, "%Y-%m-%d")).days * 24 * 60 * 60
    ),
    features=[
        Feature(name="CRIM", dtype=ValueType.FLOAT),
        Feature(name="ZN", dtype=ValueType.FLOAT),
        Feature(name="INDUS", dtype=ValueType.FLOAT),
        Feature(name="CHAS", dtype=ValueType.FLOAT),
        Feature(name="NOX", dtype=ValueType.FLOAT), 
        Feature(name="RM", dtype=ValueType.FLOAT),
        Feature(name="AGE", dtype=ValueType.FLOAT),
        Feature(name="DIS", dtype=ValueType.FLOAT),
        Feature(name="RAD", dtype=ValueType.FLOAT),
        Feature(name="TAX", dtype=ValueType.FLOAT),
        Feature(name="PTRATIO", dtype=ValueType.FLOAT),
        Feature(name="B", dtype=ValueType.FLOAT),
        Feature(name="LSTAT", dtype=ValueType.FLOAT),
    ],
    online=True,
    input=feature_source,
    tags={}
    )

In [128]:
#%%bash
#cd ../features
#feast apply

In [188]:
%%cmd
cd ../features
feast apply

Microsoft Windows [Version 10.0.19043.1110]
(c) Microsoft Corporation. All rights reserved.

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\notebooks>cd ../features

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\features>feast apply
Registered entity ID
Registered feature view house_price_details
Deploying infrastructure for house_price_details

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\features>

In [189]:
import pandas as pd
from feast import FeatureStore

In [218]:
house_price_id = [1, 2, 33]
now = datetime.now()
timestamps = [datetime(now.year, now.month, now.day)] *len(house_price_id)
entity_df = pd.DataFrame.from_dict({"ID": house_price_id, "event_timestamp": timestamps})
entity_df.head()

,ID,event_timestamp
0,1,2076-07-23
1,2,2076-07-23
2,33,2076-07-23


In [219]:
#Get historical features
store = FeatureStore(repo_path=Path(config.BASE_DIR, "features"))
training_df = store.get_historical_features(
    entity_df=entity_df,
    feature_refs=[
        "house_price_details:CRIM", 
        "house_price_details:INDUS"
        ],
).to_df()
training_df.head()

,event_timestamp,ID,house_price_details__CRIM,house_price_details__INDUS
0,2076-07-23 00:00:00+00:00,1,0.02731,7.07
1,2076-07-23 00:00:00+00:00,2,0.02729,7.07
2,2076-07-23 00:00:00+00:00,33,1.15172,8.14
